# Importar Librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages as pdf
import numpy as np
import win32com.client as win32
import sys
import requests 
import urllib.request
import json
import importlib
import os
import datetime as dt
import OleFileIO_PL
import xlrd

base_path = os.getcwd()
sys.path.append(base_path + '//..')

import Cumminsaceite as cu
#connection=cu.Connection('camilo.rodriguez.lazo@cummins.cl','camilo2021')
connection=cu.Connection("mauricio.vera@cummins.cl","ctJy.S8Juq8F")##Agregar usuario y clave
# connection=cu.Connection('wilson.soriano@cummins.cl','N<46MfH~-MrF')
connection.getToken()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Utilidades

In [2]:
def ruta_descarga(idFaena, desde,hasta):
    # Descarga de pagina Scaa (datos mas recientes)
    # https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=11&desde=01/01/2023&hasta=10/01/2023&flota=-1&alerta=1&componente=M09
    return "https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=" + idFaena + "&desde=" + desde +"&hasta=" +hasta + "&flota=-1&alerta=1&componente=M09"

# Parametros

In [3]:
# Se guardara en la carpeta '02_Archivos' 
carpeta_scaa = 'DatosScaa'
carpeta_limites = 'DatosLimites'

# Fechas de descarga de datos
desde = '01/01/2023'
hasta = dt.date.today().strftime('%d/%m/%Y')

config_path = '..\\'

carpeta_scaa = os.path.abspath(base_path + '/../../02_Archivos/' + carpeta_scaa)
carpeta_limites = os.path.abspath(base_path + '/../../02_Archivos/' + carpeta_limites)

if carpeta_scaa:
        # carpeta_guardado = carpeta_guardado + '\\'
        # directory = save_path + '\\' + carpeta_guardado 
        if not os.path.exists(carpeta_scaa):
            os.makedirs(carpeta_scaa)

horometro_path = 'Z:\\Share_DCC$\\Ingenieria y Procesos\\Bases de datos\\HOROMETRO\\consolidado_horometro.csv'
nomenclatura_path = ""






## Limites Condenatorios

In [4]:
# leer unidades de elementos
f = open(carpeta_limites +'\\unidades.json')
unidades = json.load(f)
f.close()

In [5]:
# Leer limites condenatorios
limites = pd.read_csv(carpeta_limites +'\\limites.csv')

## Datos SCAA

In [6]:
f = open(config_path +'config.json')
config = json.load(f)
f.close()

In [7]:
for i,(faena, indice) in enumerate(config.items()):
    print(faena, indice)

faenas {'Andina': 11, 'Antucoya': 124, 'Caserones': 67, 'Cerro Negro Norte': 66, 'Lomas Bayas': 27, 'Los Pelambres': 15, 'Collahuasi': 14, 'Chuquicamata': 13, 'Gaby': 53, 'Ministro Hales': 63, 'Esperanza': 121, 'Esperanza Sur': 127, 'Qblanca': 16, 'Radomiro Tomic': 18, 'Sierra Gorda': 65, 'SPENCE': 125}
updates {'last_date_scaa': '27/06/2023'}


In [8]:

for i,faena in enumerate(['Cerro Negro Norte','Lomas Bayas']):

    # Extraccion datos de plataforma scaa
    fG = urllib.request.urlopen(ruta_descarga(str(config[faena]),desde, hasta))

    # Guardado de datos en archivo excel
    fichero = open(carpeta_scaa + '\\temp.xls',"wb+")
    fichero.write(fG.read())
    fichero.close()

    # Abrir archivo excel descargado
    scaa_temp = pd.read_excel(carpeta_scaa + '\\temp.xls', decimal=",", sheet_name=0, skiprows=3)
    scaa_temp=scaa_temp.iloc[:,1:]

    scaa_faena = scaa_temp.copy()

    # Columnas a utilizar
    scaa_faena = scaa_faena[['Nro.Muestra','Faena','Flota','Modelo','Equipo','Componente','Posición','Fecha Muestra','Horas Aceite',
        'Fecha Cambio Aceite','Intervalo Cambio de Aceite','Desviación en mantención','Visc.40','Visc 100', 
        'Diluc.','Agua','TBN','Hollin','Oxid.','Nitrac.','Sulf.','Ind.PQ.','AL','B','CA','CR','CU',
        'FE','K','MG','MO','NA','NI','P','PB','SI','SN','ZN','Detalle Alerta','Respuesta Alerta']]
    # filtro motores diesel
    scaa_faena = scaa_faena[scaa_faena.Componente =='Motor Diesel']
    scaa_faena.drop(columns=['Componente'],inplace=True)
    
    # crear tabla consolidada de scaa
    if i == 0:
        consolidado_scaa = scaa_faena.copy()
    else:
        consolidado_scaa = pd.concat([consolidado_scaa, scaa_faena], ignore_index=True, sort=False)


KeyError: 'Cerro Negro Norte'

In [ ]:
consolidado_scaa.to_excel(carpeta_scaa + '\\consolidado_scaa_subgrupo.xlsx')

In [ ]:
import  aspose.cells 
from aspose.cells import Workbook
workbook = Workbook(carpeta_scaa + '\\temp.xls')
workbook.save(carpeta_scaa + '\\temp2.xls')